In [ ]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [20]:
import numpy as np
import tensorflow as tf
import read_mnist_data as rmd

# read dataset from disk
DATA_FOLDER = '/home/chengming/TensorFlow-Examples/data/'
TRAIN_IMAGES_SET = 'train-images-idx3-ubyte.gz'
TRAIN_LABELS_SET = 'train-labels-idx1-ubyte.gz'
TEST_IMAGES_SET = 't10k-images-idx3-ubyte.gz'
TEST_LABELS_SET = 't10k-labels-idx1-ubyte.gz'
validation_size = 5000
with open(DATA_FOLDER + TRAIN_IMAGES_SET, 'rb') as f:
    train_images = rmd.extract_images(f)
with open(DATA_FOLDER + TRAIN_LABELS_SET, 'rb') as f:
    train_labels = rmd.extract_labels(f, one_hot = True)
with open(DATA_FOLDER + TEST_IMAGES_SET, 'rb') as f:
    test_images = rmd.extract_images(f)
with open(DATA_FOLDER + TEST_LABELS_SET, 'rb') as f:
    test_labels = rmd.extract_labels(f, one_hot = True)
#validation_images = train_images[:validation_size]
#validation_labels = train_labels[:validation_size]
#train_images = train_images[validation_size:]
#train_labels = train_labels[validation_size:]

train_shape = train_images.shape
test_shape = test_images.shape
train_images = train_images.reshape(train_shape[0], train_shape[1]*train_shape[2])
test_images = test_images.reshape(test_shape[0], test_shape[1]*test_shape[2])


print train_images.shape
print train_labels.shape, '\t', train_labels[0][0:6], train_labels[0][6:]
print test_images.shape
print test_labels.shape, '\t', test_labels[0]

('Extracting', '/home/chengming/TensorFlow-Examples/data/train-images-idx3-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/train-labels-idx1-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/t10k-images-idx3-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/t10k-labels-idx1-ubyte.gz')
(60000, 784)
(60000, 10) 	[ 0.  0.  0.  0.  0.  1.] [ 0.  0.  0.  0.]
(10000, 784)
(10000, 10) 	[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]


In [21]:
# Parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = train_images.shape[1] # MNIST data input (img shape: 28*28)
n_classes = train_labels.shape[1] # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [22]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [23]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
# tf.nn.softmax_cross_entropy_with_logits(logits, labels) : 
#     Computes softmax cross entropy between logits and labels
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [24]:
# Launch the graph
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        #total_batch = int(mnist.train.num_examples/batch_size)
        total_batch = int(train_images.shape[0] / batch_size)
        start_idx = 0
        # Loop over all batches
        for i in range(total_batch):
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            batch_x = train_images[start_idx:start_idx+batch_size, :]
            batch_y = train_labels[start_idx:start_idx+batch_size, :]
            start_idx += batch_size
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: test_images, y: test_labels})

Epoch: 0001 cost= 34251.508436076
Epoch: 0002 cost= 9520.380238953
Epoch: 0003 cost= 5972.234848231
Epoch: 0004 cost= 4144.091943140
Epoch: 0005 cost= 3005.272052015
Epoch: 0006 cost= 2211.064555550
Epoch: 0007 cost= 1650.982919329
Epoch: 0008 cost= 1233.027562503
Epoch: 0009 cost= 924.394988686
Epoch: 0010 cost= 699.312035172
Epoch: 0011 cost= 532.900578481
Epoch: 0012 cost= 417.552393979
Epoch: 0013 cost= 332.261287769
Epoch: 0014 cost= 257.579251701
Epoch: 0015 cost= 216.932627063
Epoch: 0016 cost= 166.336190315
Epoch: 0017 cost= 174.897443741
Epoch: 0018 cost= 151.183051023
Epoch: 0019 cost= 124.570151967
Epoch: 0020 cost= 95.250273969
Epoch: 0021 cost= 113.916530010
Epoch: 0022 cost= 86.459675155
Epoch: 0023 cost= 86.561664333
Epoch: 0024 cost= 84.887135433
Epoch: 0025 cost= 73.595128405
Epoch: 0026 cost= 79.953916192
Epoch: 0027 cost= 67.212837030
Epoch: 0028 cost= 75.587888081
Epoch: 0029 cost= 59.016245907
Epoch: 0030 cost= 75.250701538
Optimization Finished!
Accuracy: 0.9577
